# Importing the required resources

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import ast
import string
from google.colab import files
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install inltk

!pip install indic-nlp-library
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git

!pip install stanza

import inltk
import stanza

     |████████████████████████████████| 81kB 3.0MB/s 
     |████████████████████████████████| 1.3MB 8.6MB/s 
     |████████████████████████████████| 235kB 16.8MB/s 
     |████████████████████████████████| 1.1MB 19.9MB/s 
     |████████████████████████████████| 296kB 21.2MB/s 
     |████████████████████████████████| 143kB 24.7MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-cp36-none-any.whl size=26307 sha256=baa8569a30bb7c2ebf213b7ce24c817db1e41960e9f745a087aa4413f890a312
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=1c1226f5a23340b84c8aff04458c1a7aea2dcc86b3e85f180db12b7bfe26d030
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built typing idna-ssl
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fas

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 18.49 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Checking out files: 100% (28/28), done.
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 1266 (delta 47), reused 51 (delta 24), pack-reused 1178
Receiving objects: 100% (1266/1266), 9.56 MiB | 5.92 MiB/s, done.
Resolving deltas: 100% (651/651), done.
     |████████████████████████████████| 235kB 2.7MB/s 


In [ ]:
#INDIC NLP
from indicnlp.tokenize import indic_tokenize

#STANFORD NLP
stanza.download('hi') #208 MB file, will take a long time to execute

2020-10-29 08:21:15 INFO: Downloading default packages for language: hi (Hindi)...
2020-10-29 08:21:27 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
# First add the folder at https://drive.google.com/drive/folders/11tcWX3jDWzSLkGhduIonhXpwYMOXfuWQ?usp=sharing to your personal "My Drive"
train_data = pd.read_csv("/content/gdrive/My Drive/CS685/constraint_hindi_train.csv")
valid_data = pd.read_csv("/content/gdrive/My Drive/CS685/constraint_hindi_valid.csv")

In [ ]:
# The output files for this code are present here: 
# https://drive.google.com/file/d/1hd5zbNqbohhM0b7wzhkvnfpHc-lm06Pp/view?usp=sharing
# Add this to the CS685 folder in your My Drive where you had saved the train and test data
processed_df_train = pd.read_csv("/content/gdrive/My Drive/CS685/processed_df_train.csv")
processed_df_valid = pd.read_csv("/content/gdrive/My Drive/CS685/processed_df_valid.csv")

,UID,post,tokenised_indic,labels,filtered_indic,lemmatized_posts,filtered_lemmatized_indic
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्क...,"['मेरे', 'देश', 'के', 'हिन्दु', 'बहुत', 'निराल...","hate,offensive","['मेरे', 'देश', 'हिन्दु', 'निराले', 'पक्के', '...",मैं देश का हिन्दु बहुत निराला है कुछ तो पक्का ...,"['मैं', 'देश', 'हिन्दु', 'निराला', 'पक्का', 'र..."
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,"['सरकार', 'हमेशा', 'से', 'किसानों', 'की', 'कमा...",non-hostile,"['सरकार', 'हमेशा', 'किसानों', 'कमाई', 'बढ़ाने'...",सरकार हमेशा से किसान का कमाई को बढ़ा का लिए नई...,"['सरकार', 'हमेशा', 'किसान', 'कमाई', 'बढ़ा', 'न..."
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...","['सुशांत', 'ने', 'जो', 'बिजनेस', 'डील', '9', '...",non-hostile,"['सुशांत', 'बिजनेस', 'डील', '9', 'जून', 'वो', ...",सुशांत ने जो बिजनेस डील 9 जून को कर था COMMA व...,"['सुशांत', 'बिजनेस', 'डील', '9', 'जून', 'COMMA..."
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"['साले', 'जेएनयू', 'छाप', 'कमिने', 'लोग', 'हिन...","defamation,offensive","['साले', 'जेएनयू', 'छाप', 'कमिने', 'लोग', 'हिन...",साला जेएनयू छाप कमिने लोग हिंदू को यह कह है का...,"['साला', 'जेएनयू', 'छाप', 'कमिने', 'लोग', 'हिं..."
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,"['#', 'unlock4guidelines', '-', 'अनलॉक', '-', ...",non-hostile,"['#', 'unlock4guidelines', 'अनलॉक', '4', 'गाइड...",#unlock4guidelines - अनलॉक-4 का लिए गाइडलाइन ज...,"['#', 'unlock4guidelines', 'अनलॉक', '4', 'गाइड..."


## Output data description

* **post:** original post from training data

* **labels:** taken from training data

* **tokenised_indic:** tokenised posts using indic nlp library

* **filtered_indic:** took posts from tokenised_indic and removed stop words and excess punctuation

* **lemmatized_posts:** lemmatized the original posts using stanford nlp

* **filtered_lemmatized_indic:** lemmatized original posts, then tokenised them and removed stop words and excess punctuation

The code below is only for testing out data preprocessing. Use the dataframe generated above if you want to train models

In [ ]:
#Tweet manipulation - data preprocessing

#Remove emojis
def remove_emojis(tweet):
  emojis = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
          u"\U00002500-\U00002BEF"  # chinese char
          u"\U00002702-\U000027B0"
          u"\U00002702-\U000027B0"
          u"\U000024C2-\U0001F251"
          u"\U0001f926-\U0001f937"
          u"\U00010000-\U0010ffff"
          u"\u2640-\u2642"
          u"\u2600-\u2B55"
          u"\u200d"
          u"\u23cf"
          u"\u23e9"
          u"\u231a"
          u"\ufe0f"  # dingbats
          u"\u3030"
                            "]+", flags=re.UNICODE)
  return emojis.sub(r'', tweet)

#Remove URLs and the 'RT' string
def remove_url_and_RT(tweet):
  return re.sub(r"http\S+|RT", "", tweet)

#Remove usernames
def remove_username(tweet):
  return re.sub(r"@\S+","",tweet)

#Remove some punctuation
def remove_punct(tweet):
  tweet.translate(str.maketrans('', '', string.punctuation))
  return re.sub(r"\n|।|\|", " ", tweet)

In [ ]:
# Tokenization + Removing basic punctuation
def tokenize(dataset):
  n = int(dataset.size/3)
  processed = []
  for i in range(n):
    post = dataset.loc[i]["Post"]
    post = remove_punct(remove_username(remove_url_and_RT(remove_emojis(post))))
    tokenised_indic = indic_tokenize.trivial_tokenize(post)
    processed.append([dataset.loc[i]["Unique ID"], post, tokenised_indic, dataset.loc[i]["Labels Set"]])

  return processed

processed_df = pd.DataFrame(tokenize(valid_data), columns = ['UID', 'post','tokenised_indic','labels'])

In [ ]:
# Loads list of hindi stopwords
# Source: https://github.com/taranjeet/hindi-tokenizer/blob/master/stopwords.txt
# Add to CS685 folder in My Drive where you saved your data
f = open("/content/gdrive/My Drive/CS685/hindi_stopwords.txt")
lines = f.readlines()
stopwords = []
for line in lines:
  stopwords.append(line.strip('\n'))

['अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'अंदर', 'आदि', 'आप', 'इत्यादि', 'इन ', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर', 'करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 'दूसरे', 'दो', 'द्वारा', 'न', 'नके', 'नहीं', 'ना', 'निहायत', 'नीचे', 'ने', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रह

In [ ]:
# Filters stopwords out from Indic-NLP tokenized representations and saves them to dataframe
# Then filters out extra punctuation

extra_punctuation = [',', '.', '-', ':', "'", '_', '"', ";", '(', ')'] # keeping hashtags, exclamation and question marks for further analysis
filterer = stopwords + extra_punctuation

filtered_posts = []
for i in range(len(processed_df)):
  list_words = processed_df.loc[i, 'tokenised_indic']
  final_list_words = [w for w in list_words if w not in filterer]
  filtered_posts.append(final_list_words)

processed_df.loc[:, 'filtered_indic'] = filtered_posts

In [ ]:
# Lemmatization on original post - Will take around 5-10 min
nlp = stanza.Pipeline(lang="hi", processors='tokenize,pos,lemma') 

lemmatized = []
for i in range(len(processed_df)):
  doc = nlp(processed_df.loc[i, 'post'])
  lemma_doc = ""
  for sent in doc.sentences:
    for word in sent.words:
      lemma_doc += word.lemma + " "
  
  lemmatized.append(lemma_doc.rstrip())

processed_df.loc[:, 'lemmatized_posts'] = lemmatized

2020-10-29 08:24:40 INFO: Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |
| pos       | hdtb    |
| lemma     | hdtb    |

2020-10-29 08:24:40 INFO: Use device: cpu
2020-10-29 08:24:40 INFO: Loading: tokenize
2020-10-29 08:24:40 INFO: Loading: pos
2020-10-29 08:24:41 INFO: Loading: lemma
2020-10-29 08:24:41 INFO: Done loading processors!


In [ ]:
# Take the lemmatized posts and tokenize them. Then remove stopwords.
filtered_lemmatized = []
for i in range(len(processed_df)):
  post = processed_df.loc[i, 'lemmatized_posts']
  post = remove_punct(remove_username(remove_url_and_RT(remove_emojis(post))))
  tokenised_indic = indic_tokenize.trivial_tokenize(post)
  final_list_words = [w for w in tokenised_indic if w not in filterer]
  filtered_lemmatized.append(final_list_words)

processed_df.loc[:, 'filtered_lemmatized_indic'] = filtered_lemmatized
processed_df.to_csv("/content/gdrive/My Drive/CS685/processed_df_valid.csv", index=False)

# Baseline RNN



In [ ]:
import torch
import torch.nn as nn
from torchtext.data import Field, BucketIterator
import torch.optim as optim


In [ ]:
def convert_labels(df):
    df[['fake','hate','offensive','defamation','non-hostile']] = 0
    for inx, row in df.iterrows():
        for label in row['labels'].split(','):
            df.loc[inx, label]=1
    return df

In [ ]:
rnn_df = convert_labels(processed_df)

In [ ]:
rnn_df.head()

,UID,post,tokenised_indic,labels,filtered_indic,lemmatized_posts,filtered_lemmatized_indic,fake,hate,offensive,defamation,non-hostile
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्क...,"['मेरे', 'देश', 'के', 'हिन्दु', 'बहुत', 'निराल...","hate,offensive","['मेरे', 'देश', 'हिन्दु', 'निराले', 'पक्के', '...",मैं देश का हिन्दु बहुत निराला है कुछ तो पक्का ...,"['मैं', 'देश', 'हिन्दु', 'निराला', 'पक्का', 'र...",0,1,1,0,0
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,"['सरकार', 'हमेशा', 'से', 'किसानों', 'की', 'कमा...",non-hostile,"['सरकार', 'हमेशा', 'किसानों', 'कमाई', 'बढ़ाने'...",सरकार हमेशा से किसान का कमाई को बढ़ा का लिए नई...,"['सरकार', 'हमेशा', 'किसान', 'कमाई', 'बढ़ा', 'न...",0,0,0,0,1
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...","['सुशांत', 'ने', 'जो', 'बिजनेस', 'डील', '9', '...",non-hostile,"['सुशांत', 'बिजनेस', 'डील', '9', 'जून', 'वो', ...",सुशांत ने जो बिजनेस डील 9 जून को कर था COMMA व...,"['सुशांत', 'बिजनेस', 'डील', '9', 'जून', 'COMMA...",0,0,0,0,1
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"['साले', 'जेएनयू', 'छाप', 'कमिने', 'लोग', 'हिन...","defamation,offensive","['साले', 'जेएनयू', 'छाप', 'कमिने', 'लोग', 'हिन...",साला जेएनयू छाप कमिने लोग हिंदू को यह कह है का...,"['साला', 'जेएनयू', 'छाप', 'कमिने', 'लोग', 'हिं...",0,0,1,1,0
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,"['#', 'unlock4guidelines', '-', 'अनलॉक', '-', ...",non-hostile,"['#', 'unlock4guidelines', 'अनलॉक', '4', 'गाइड...",#unlock4guidelines - अनलॉक-4 का लिए गाइडलाइन ज...,"['#', 'unlock4guidelines', 'अनलॉक', '4', 'गाइड...",0,0,0,0,1


# Augmentation

Inltk augmentation is outdated so will try manual augmentation.
Will start working with synonyms now.


In [ ]:
! pip install pyiwn
import pyiwn 
iwn = pyiwn.IndoWordNet(lang=pyiwn.Language.HINDI) 

2020-11-12:05:01:02,654 INFO     [helpers.py:20] Downloading IndoWordNet data of size ~31 MB...


[██████████████████████████████████████████████████]

2020-11-12:05:01:06,577 INFO     [helpers.py:43] Extracting /root/iwn_data.tar.gz into /root...


2020-11-12:05:01:07,795 INFO     [helpers.py:48] Removing temporary zip file from /root/iwn_data.tar.gz
2020-11-12:05:01:07,801 INFO     [helpers.py:51] IndoWordNet data successfully downloaded at /root/iwn_data
2020-11-12:05:01:07,805 INFO     [iwn.py:43] Loading hindi language synsets...
2020-11-12:05:01:08,723 INFO     [utils.py:141] NumExpr defaulting to 2 threads.


In [ ]:
from ast import literal_eval

Augmentation example:

In [ ]:
augmented2 = []
for i in range(len(processed_df_train)):
    augmented2.append([processed_df_train.loc[i,'UID'],processed_df_train.loc[i,'post'],processed_df_train.loc[i,'labels']])

    words = list(filter(('').__ne__, processed_df_train.loc[i,'post'].split(' ')))
    words_new = list(filter(('').__ne__, processed_df_train.loc[i,'post'].split(' ')))
    
    '''
    Augmentation style 2 where the count is scaled according to the original frequency:
    '''
    # if processed_df_train.loc[i,'labels'].find('fake') is not -1:
    #     count = 2
    # elif (processed_df_train.loc[i,'labels'].find('hate') is not -1) or (processed_df_train.loc[i,'labels'].find('offensive') is not -1):
    #     count = 3
    # elif processed_df_train.loc[i,'labels'].find('defamation') is not -1:
    #     count = 4
    # else:
    #     count = 1

    '''
    Augmentation style 1 where the count is scaled to 2 for all classes i.e. we double the dataset
    '''
    count = 2

    for j in range(1,count):
        for k in range(len(words)):
            try:
                iwn.synsets(words[k], pos=pyiwn.PosTag.NOUN)[0]
            except:
                continue
            
            synonyms = iwn.synsets(words[k], pos=pyiwn.PosTag.NOUN)[0].lemma_names() #Finding all synonyms
            if (len(synonyms)>j):
                words_new[k] = synonyms[j] #replacing the word

        new = ''
        for word in words_new:
            new = new + word + ' '
        augmented2.append([processed_df_train.loc[i,'UID'],new,processed_df_train.loc[i,'labels']])

In [ ]:
augmented2_df = pd.DataFrame(augmented2, columns = ['UID', 'post','labels'])

,UID,post,labels
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्क...,"hate,offensive"
1,1,मेरे राष्ट्र के हिन्दु कई निराले है कुछ तो पक्...,"hate,offensive"
2,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
3,2,शासन हमेशा से किसानों पेंट आमदनी को बढ़ाने के ...,non-hostile
4,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
...,...,...,...
11451,5726,जॉब गंवा चुके दोस्त पेंट मदद: नक़दी के बजाए भे...,non-hostile
11452,5727,बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क न...,fake
11453,5727,पश्चिम बंगाल मेंमें हिन्दू मरे हैं इसलिए मुझे ...,fake
11454,5728,रक्षा मंत्री ने पूर्व राष्ट्रपति #PranabMukhe...,non-hostile


In [ ]:
augmented2_df.to_csv("/content/gdrive/My Drive/CS685/augmented2_train.csv", index=False)